# Modelo hidrológico
En este Notebook vamos a crear un modelo hidrologico y vamos a utilizar widgets interactivos para intentar entender mejor como funciona el modelo, como influencian los parametros del modelo en el resultado y como podemos calibrar el modelo. 

Primero de todo, ¿que es un modelo hidrologico? Imagina que queremos predecir el hidrograma de un rio que recoje el agua de una cuenca cuando llueve. Para este proposito utilizamos un modelo hidrologico, que no es mas que un conjunto de ecuaciones que describen de una manera simplificada los procesos hidrologicos que ocurren en la cuenca. Estas ecuaciones incluyen diferentes parametros que describen algunas de las propiedades de la cuenca, por ejemplo las caracterisiticas del suelo.

<left><img src="util/diagrama_cuenca.gif" width="500px">

En este ejemplo, vamos a utilizar un modelo simple (una adaptacion del modelo [HyMOD](https://doi.org/10.1002/9781118665671.ch14)) que tiene 5 parametros:

- **Capacidad del suelo** (mm): capacidad que tiene el suelo de retener agua de lluvia
- **Tasa de evaporacion**: ratio de evaporacion o proporcion de lluvia que se evapora
- **Tasa de infiltracion**: ratio de infiltracion proporcion del agua de lluvia efectiva (que no se evapora) que se infiltra en el suelo.
- **Tiempo de viaje - Flujo superficial** (dias): tiempo de concentracion del agua superficial o tiempo que tarda el agua superficial en llegar a la desembocadura de la cuenca.
- **Tiempo de viaje - Flujo subterraneo** (dias): tiempo de concentracion del agua subterranea o tiempo que tarda el agua subterranea en llegar a la desembocadura de la cuenca.

En la imagen de abajo podemos ver como el modelo representa los procesos hidrologicos para asi obtener finalmente la prediccion del hidrograma del rio para los proximos meses. Como puedes ver el suelo es presentado como un deposito con una cierta capacidad que si es sobrepasada desborda generando flujo superficial y que tambien desaloja agua por su parte inferior generando agua subterranea.

<left><img src="util/Diagrama_modelo.gif" width="900px">
    
Vamos a crear el modelo y probarlo para comprender mejor como funciona.

Lo primero como siempre es importar las librerias de funciones que vamos a necesitar
    
### Definicion del modelo

In [2]:
# importamos las librerias necesarias
import numpy as np; import pandas as pd; import matplotlib.pyplot as plt
from ipywidgets import interact

def hyd_model(capacidad_suelo, tasa_evaporacion, tasa_infiltracion, t_superficial, t_subterraneo):
    
    #######################################################################
    # Initialization of variables
    #######################################################################
    effec_rain = np.zeros((T,1)) # Lluvia efectiva [mm/t]
    et          = np.zeros((T,1)) # Tasa de evapotranspiracion [mm/t]
    sm          = np.zeros((T+1,1)) # Contenido de humdedad en el suelo [mm] (suponemos que el suelo esta seco inicialmente)
    sL          = np.zeros((T+1,1)) # Slow reservoir moisture content [mm]
    sF          = np.zeros((T+1,1)) # Fast reservoir moisture content [mm]
    q_sub       = np.zeros((T,1)) # Flujo subterraneo [mm/t]
    q_sur       = np.zeros((T,1)) # Flujo superficial [mm/t]
    
    for t in range(1,T):

        ##### Lluvia efectiva #####
        effec_rain[t] = max(sm[t-1] + prec[t] - capacidad_suelo, 0)

        ##### Humedad de suelo temporal #####
        sm_temp = max(min(sm[t-1] + prec[t], capacidad_suelo), 0)

        ##### Evapotranspiracion #####
        W = min(np.abs(sm_temp/capacidad_suelo)*tasa_evaporacion, 1) # Factor de correccion de la evapotranspiracion
        et[t] = W * etp[t] # Calculo de la evapotranspiracion

        ##### Humedad del suelo #####       
        sm[t] = max(min(sm[t-1] + prec[t] - et[t], capacidad_suelo), 0)

        ##### Flujo subterraneo ######
        sL[t] = sL[t-1] + tasa_infiltracion*effec_rain[t] - q_sub[t-1]
        q_sub[t] = 1/t_subterraneo * sL[t]

        ##### Flujo superficial #####
        sF[t] = sF[t-1] +  (1-tasa_infiltracion)*effec_rain[t] - q_sur[t-1]
        q_sur[t] = 1/t_superficial * sF[t]

    ##### Flujo total #####
    A = 500 * 10000  # ha to m2
    Q_sim = (q_sur + q_sub) * 0.001 * A
    
    return Q_sim

## Cargar datos observados de lluvia, evapotranspiracion y caudal
Cargamos los datos de precipitación diaria (mm/día), evapotranspiración potencial (mm/día) y flujo de salida (m3/día) observados de 2010 a 2019. Para ello usamos archivo Excel que vamos a cargar y guardar como un dataframe de Pandas.

In [3]:
obs_data = pd.read_excel('Datos/data example 2.xlsx',index_col = 'date')
year_of_study = 2012
obs_data_year = obs_data[obs_data.index.year == year_of_study] # para seleccionar solo un año
T = len(obs_data_year) # numero de pasos de tiempo
# Inputs
dates = obs_data_year.index
prec  = obs_data_year['rain']
etp   = obs_data_year['etp']
Q_obs = obs_data_year['outflow']

## Calibracion del modelo hidrologico
Un modelo hidrologico suele tener un gran número de parámetros. El usuario es quien decide el valor de estos parámetros para una aplicación particular en función de la información y datos que tengamos sobre los parámetros. Normalmente no vamos a tener mediciones directas de los valores de dichos parametros pero si es probable que tengamos datos climaticos historicos y del caudal del rio (hidrograma historico del rio). Con estos datos podemos inferir los valores de los parámetros encontrando los valores que hacen que el resultado del modelo se ajuste mejor al hidrograma historico del rio, esto se denomina **calibración** del modelo.

La forma más sencilla de hacer esto es cambiando los valores de los parámetros de uno en uno y observando cómo esto cambia el resultado del modelo y como de bien se ajusta al hidrograma historico. Esto es similar a lo que hicimos en el Notebook [Ejemplo - Onda de Sonido](Ejemplo%20-%20Onda%20de%20Sonido.ipynb), solo que ahora no simulamos una onda de sonido sino el caudal de un rio y ahora no tenemos 3 parametros (amplitud, fase, frecuencia) sino 5 parametros.

### Bondad de ajuste
Para medir si los valores simulados con un modelo están cerca de los valores de los datos observados, utilizamos indicadores de ajuste. Un ejemplo es el Error cuadrático medio (RMSE). El RMSE es la raíz cuadrada de la varianza de los residuos. Indica el ajuste absoluto del modelo a los datos: qué tan cerca están los puntos de datos observados de los valores predichos del modelo. Tiene la propiedad de estar en las mismas unidades que la variable de respuesta. Valores más bajos de RMSE indican un mejor ajuste.

$RMSE = \sqrt{(\frac{1}{n})\sum_{i=1}^{n}(sim_{i} - obs_{i})^{2}}$

In [4]:
def RMSE_function(obs,sim):
    
    RMSE = np.sqrt(((sim - obs) ** 2).mean())
    return RMSE

def NSE_function(obs,sim):
    NSE = (1 - (np.sum((sim - obs) ** 2) / np.sum((obs - np.mean(obs))**2)))
    
    return NSE

Intenta ahora, cambiando los valores de los 5 parametros, calibrar el modelo, es decir ajustar el hidrograma que simula el modelo al hidrograma historico.

In [5]:
@interact(capacidad = (5, 90, 1), tasa_evap = (0.05, 1, 0.05), tasa_infil = (0, 1, 0.05), 
          tiempo_sup = (0.8,2,0.1), tiempo_sub = (2,30,1))

def modelo_hidrologico_v1(capacidad=5, tasa_evap=0.05, tasa_infil=0.0, tiempo_sup=0.8, tiempo_sub=2):
    
    sim_data_year = pd.DataFrame(0, index=obs_data_year.index, columns = ['rain','outflow'])
    sim_data_year['rain'] = obs_data_year['rain']
    
    # Simulacion
    Q_sim = hyd_model(capacidad, tasa_evap, tasa_infil, tiempo_sup, tiempo_sub)
    sim_data_year['outflow'] = Q_sim
    
    RMSE = RMSE_function(Q_obs,sim_data_year['outflow'])

    # Visualizacion de resultados  
    plt.figure(figsize=(15,5))
    plt.plot(dates,Q_obs, label = 'obs', color = 'black')
    plt.plot(dates,Q_sim, label = 'sim', color = 'darkblue',alpha = 0.6)
    plt.ylim(0,200000)
    plt.title('Root mean squared error: '+str(round(RMSE)),fontsize = 15)
    plt.legend()
    plt.ylabel('m3/day')
    plt.show()

interactive(children=(IntSlider(value=5, description='capacidad', max=90, min=5), FloatSlider(value=0.05, desc…

Comprender el proceso de calibración de un modelo le ayuda a comprender el modelo en sí. Qué significa cada parámetro, qué parámetros influyen más en el resultado y comprobar que el modelo se comporta de forma lógica. Y con elementos interactivos como widgets facilitamos esta tarea y por tanto la comprensión del modelo (por ejemplo hidrológico)